
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# 02 - Deploy a Simple DAB

### Estimated Duration: 15-20 minutes

In this lab, you will:

1. Create a simple job using the UI.

2. Examine its YAML configuration.

3. Use the YAML configuration to update the **databricks.yml** configuration file.

4. Use the Databricks CLI to run the following DAB commands:
    - Validate the job
    - Deploy the job
    - Run the job


## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:

1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

  - In the drop-down, select **More**.

  - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

1. Find the triangle icon to the right of your compute cluster name and click it.

1. Wait a few minutes for the cluster to start.

1. Once the cluster is running, complete the steps above to select your cluster.

## A. Classroom Setup

Run the following cell to configure your working environment for this course.

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course.

In [0]:
%run ../Includes/Classroom-Setup-02L

## IMPORTANT LAB INFORMATION

Recall that your credentials are stored in a file when running [0 - REQUIRED - Course Setup and Authentication]($../0 - REQUIRED - Course Setup and Authentication).

If you end your lab or your lab session times out, your environment will be reset.

If you encounter an error regarding unavailable catalogs or if your Databricks CLI is not authenticated, you will need to rerun the [0 - REQUIRED - Course Setup and Authentication]($../0 - REQUIRED - Course Setup and Authentication) notebook to recreate the catalogs and your Databricks CLI credentials.

**Use classic compute to use the CLI through a notebook.**

## SCENARIO

You are in charge of deploying Databricks projects for your CI/CD pipelines in your organization using Databricks Asset Bundles. A coworker has sent you a notebook located in **./src/our_project_code**, and they want your help to begin the CI/CD process to deploy their project.

To start, let's first deploy the project to the development environment. Your task is to retrieve the YAML job configuration and update the **databricks.yml** bundle file located in the **02L - Deploy a Simple DAB** folder. The notebook given to you simply creates a simple bronze table and silver table using the development data **nyctaxi_raw** within your **username_1_dev** catalog. The dev catalog will be specified within the job parameters.

Follow the instructions below to complete your task.


## B. Preview the Development Data
Preview the **nyctaxi_raw** data in your **username_1_dev** catalog. Notice that the development data contains a small sample of the production data (100 rows).

In [0]:
spark.sql(f'''
SELECT *
FROM {DA.catalog_dev}.default.nyctaxi_raw
''').display()

## C. TO DO: STEPS



1. Run the following cell to obtain your cluster id for the lab. You will need this for the YAML configuration.

    **NOTE:** By default, if you create the job using the UI and specify the required cluster, the cluster ID will already by shown in the YAML configuration. You can also obtain it using the code below.

In [0]:
spark.conf.get("spark.databricks.clusterUsageTags.clusterId")

2. Get the YAML configuration for the job to deploy the notebook **./src/our_project_code** by creating the job using the UI.

    **HINT:** The best way to get the necessary YAML job configuration is to manually create the job and copy the configuration. You can also refer to the [Databricks REST API reference](https://docs.databricks.com/api/workspace/introduction) for additional help.

    The job has the following requirements:

   - Name the job **lab02_job_yourfirstname**.

   - Create a job with a single task named **create_nyc_tables** using the **./02L - Deploy a Simple DAB/src/our_project_code** notebook.

   - Use your current lab cluster for the job's compute (selecting your lab compute cluster will automatically return the cluster ID in the YAML configuration).

   - Add two **Job parameters** (make sure these are for the job, not the task):

     - **catalog_name** referencing the catalog shown in **Step A**.
     
     - **display_target** with the value **Development**.

   - (Optional) Under **Job notifications**, send an email notification to yourself when the **Job** completes successfully (ensure the notification is for the job, not the task).

3. Once you have created the job, copy the YAML configuration.

    **NOTE:** You can also test the configuration by running the job if you would like.

4. Modify the **databricks.yml** configuration file for your Databricks Asset Bundle (DAB) in the folder **02 - Deploy a Simple DAB**. Complete the following steps:

   - Add the job YAML configuration from the previous step under the RESOURCES comment in the **databricks.yml** file. In the job configuration:

     - Ensure the correct extension for the notebook is included (**HINT:** The notebook is an SQL notebook).
     
     - Modify the notebook path to use a relative path. (**HINT: ./src/our_project_code.sql**)
     


   - In the **target** mapping, create a target environment called **dev** with the following configurations:
     
     - Set **dev** as the default target.
     
     - Set it to development mode.
     
     - For **Workspace** within **targets**, add the following:
       
```yaml
workspace:
  root_path: /Workspace/Users/${workspace.current_user.userName}/.bundle/${bundle.name}/${bundle.target}
```

**HINT:** View the Databricks Asset Bundles [Mappings](https://docs.databricks.com/en/dev-tools/bundles/settings.html#mappings) documentation for additional information. An example solution to the **databricks.yml** file can be found in the accompanying **solution** folder.


<br></br>
### NOTE - PLEASE READ!

Starting December 20, 2024, the [default format for new notebooks is now IPYNB (Jupyter) format](https://docs.databricks.com/en/release-notes/product/2024/december.html#the-default-format-for-new-notebooks-is-now-ipynb-jupyter-format). This may cause issues when referencing notebooks with DABs, as you must specify the file extension.

For the purpose of this course, all notebooks will be in either **.py** or **.sql** format. However, to confirm the file extension of a notebook, complete the following steps:

- In the top navigation bar, below the notebook name, select **File**.

- Scroll down and find the **Notebook format** option, then select it.

- Here, you should see the notebook format listed as **Source (.scala, .py, .sql, .r)**.


## Deploy the DAB to the Development Environment

1. Run the Databricks CLI command below to confirm the Databricks CLI is authenticated.

<br></br>
##### DATABRICKS CLI ERROR TROUBLESHOOTING:
  - If you encounter an Databricks CLI authentication error, it means you haven't created the PAT token specified in notebook **0 - REQUIRED - Course Setup and Authentication**. You will need to set up Databricks CLI authentication as shown in that notebook.

  - If you encounter the error below, it means your `databricks.yml` file is invalid due to a modification. Even for non-DAB CLI commands, the `databricks.yml` file is still required, as it may contain important authentication details, such as the host and profile, which are utilized by the CLI commands.

![CLI Invalid YAML](../Includes/images/databricks_cli_error_invalid_yaml.png)

In [0]:
%sh
databricks catalogs list

2. Check the version of the Databricks CLI. Confirm that the version is **v0.240.0**.

In [0]:
%sh
databricks -v

3. Use the `ls` command to view the available files in the current directory. Confirm that you see the **databricks.yml** file, the **src** folder, and the **02L - Deploy a Simple DAB** notebook.

In [0]:
%sh
ls

4. Validate your **databricks.yml** bundle configuration file using the Databricks CLI. Run the cell and confirm that the validation was successful. If there is an error, fix it.

    **HINT:** You can refer to the documentation for the [bundle command group](https://docs.databricks.com/en/dev-tools/cli/bundle-commands.html) for help with validating, deploying, running, and destroying a bundle.

In [0]:
%sh
databricks bundle validate

5. Deploy the bundle using the development environment target with the Databricks CLI.

    After the cell completes, manually check to see if the job was created successfully. The job name will be **[dev user_name] lab02_job_firstname**.

    Specifically:
    - check the task and make sure the notebook is referenced correctly
    - check the job parameters and confirm they are referencing the correct catalog

    **NOTE:** This will take about a minute to complete.

    **HINT:** You can refer to the documentation for the [bundle command group](https://docs.databricks.com/en/dev-tools/cli/bundle-commands.html) for help with validating, deploying, running, and destroying a bundle.


In [0]:
%sh
databricks bundle deploy -t dev

6. Run the bundle using the target development environment with the Databricks CLI.

    **NOTE:** This will take about 1-2 minutes to complete.

    **HINT:** You can refer to the documentation for the [bundle command group](https://docs.databricks.com/en/dev-tools/cli/bundle-commands.html) for help with validating, deploying, running, and destroying a bundle.

    **HINT:** Remember to use the key name from the resources mapping in the **databricks.yml** file (your name will differ):

```
resources:
  jobs:
    lab02_job_yourfirstname:    # <--- The job key name here
      name: lab02_job_yourfirstname
```


In [0]:
%sh
databricks bundle run -t dev lab02_job_peter

7. After the job successfully completes, run the following cell to confirm that the bronze table from your job was created correctly.

**NOTE:** The job created two tables in your **username_1_dev** catalog, **nyctaxi_bronze** and **nyctaxi_silver**. We will simply check the number of rows in the bronze table.

In [0]:
check_nyctaxi_bronze_table(user_catalog = DA.catalog_dev, total_count=100)

8. Lastly, destroy the bundle you created for this lab.

    **HINT:** You can refer to the documentation for the [bundle command group](https://docs.databricks.com/en/dev-tools/cli/bundle-commands.html) for help with validating, deploying, running, and destroying a bundle.


In [0]:
%sh
databricks bundle destroy --auto-approve


&copy; 2025 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>